# 3D Cloud TOA Notebook
This notebook allows the user to perform inference on cloud content in ABI data. 

ABI data is loaded from disk, and a 128x128 chip is created from user-defined lat, lon input. Users can either input one single lat/lon pair, which will be the center of the generated image, or 2 "endpoints". In the case of endpoints, the center of the image will be the midpoint of both inputs. *See the "user-defined values" cell for example usage.*

The generated image (128x128 pixel "chip") will then be generated, processed, and be fed into the model. Finally, the image and the transect used will be plotted alongside the model output, a predicted cloud mask at varying altitudes.  

## Installs/imports

In [ ]:
import os
import sys
import torch
import subprocess
from huggingface_hub import snapshot_download

In [ ]:
repo_dir = "satvision-toa"

if not os.path.exists(repo_dir):
    subprocess.run(["git", "clone", "https://github.com/nasa-nccs-hpda/satvision-toa"])
else:
    subprocess.run(["git", "-C", repo_dir, "pull"])

In [ ]:
sys.path.append('satvision-toa')
from satvision_toa.data_utils.utils_3dcloud import (
    load_abi, 
    create_chip,
    plot_rgb_chip_and_mask, 
    FCN,
    load_config
)
from satvision_toa.configs.config import _C, _update_config_from_file
from satvision_toa.models.mim import build_mim_model

## User-defined values

In [ ]:
# Time of day, year, day of year
t = 15  # 15, 16, 17, 19
YYYY = "2019"
DDD = "270"

# ----------------------------------------
# Input 2 endpoints p1, p2 or a single point to generate chip
# (single point is center of chip). Generated endpoint must be
# within the bounds:
#    lat: [-39.5, 39.5]
#    lon: [-122.5, -26.25]
# ----------------------------------------


def midpoint(p1, p2):
    return (p1[0]+p2[0])/2, (p1[1]+p2[1])/2


p1 = (20, -50)
p2 = (25, -45)
lat, lon = midpoint(p1, p2)
print(f'lat, lon midpoint used to generate chip: {lat, lon}')

## Download Demo Data From HuggingFace

In [ ]:
# Example dataset URL
hf_dataset_repo_id: str = 'nasa-cisto-data-science-group/downstream-3dclouds-subset'
abi_data_dir = snapshot_download(repo_id=hf_dataset_repo_id, allow_patterns="*.nc", repo_type='dataset')
dataset_root_path = os.path.join(abi_data_dir, 'abi-fulldisk')
dataset_root_path

In [ ]:
abi_metadata_path = os.path.join(dataset_root_path, 'ABI_EAST_GEO_TOPO_LOMSK.nc')

In [ ]:
# Root directory for all ABI data, path to NetCDF ABI file
#ROOT = "/explore/nobackup/people/jli30/data/abi_dg/abi/"
#abi_path = "/explore/nobackup/people/jgong/ABI_EAST_GEO_TOPO_LOMSK.nc"

## Read, process ABI data

In [ ]:
abi_dict = load_abi(abi_metadata_path)

## Create chip

In [ ]:
chip = create_chip(abi_dict, t, YYYY, DDD, lat, lon, dataset_root_path)

## Load model

In [ ]:
# download and update config from huggingface
config = load_config()

In [ ]:
# build base model from config, use encoder for FCN architecture
model = build_mim_model(config)
model = FCN(
    swin_encoder=model.encoder,
    num_output_channels=1, freeze_encoder=True)

In [ ]:
# load checkpoint from huggingface
cloud_checkpoint = torch.load(
    config.MODEL.RESUME, weights_only=False)
cloud_checkpoint = cloud_checkpoint['module']
cloud_checkpoint = {k.replace('model.', ''): v
                    for k, v in cloud_checkpoint.items()
                    if k.startswith('model')}

In [ ]:
# apply checkpoint to the model
model.load_state_dict(cloud_checkpoint)
model.eval()
model.cuda();

## Perform and visualize model inference

In [ ]:
pred = torch.sigmoid(model(chip))
plot_rgb_chip_and_mask(chip, pred, lat, lon)